In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns
import squarify
import plotly.graph_objs as go
from matplotlib import cbook, rc_params_from_file, rcParamsDefault
import plotly.express as px
from sklearn.metrics import roc_auc_score
from sklearn import metrics

#classifier
from sklearn.linear_model import LinearRegression, LogisticRegression 
from sklearn.metrics import mean_squared_error,r2_score, mean_absolute_error
from sklearn import svm

from sklearn.model_selection import train_test_split 
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import *
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
train = pd.read_csv(r"/kaggle/input/black-friday-sales-prediction/Train.csv")
test = pd.read_csv(r"/kaggle/input/black-friday-sales-prediction/Test.csv")

In [ ]:
print(train.shape)
train.head()

In [ ]:
train.info()

In [ ]:
print(test.shape)
test.head()

In [ ]:
test.info()

In [ ]:
#train.drop("User_ID", axis = 1, inplace = True)

#test.drop("User_ID", axis = 1, inplace = True)

In [ ]:
df = pd.concat([train, test],ignore_index=True)
print(train.shape, test.shape, df.shape)

In [ ]:
print(df.shape)
df.head()

In [ ]:
df.notnull().tail()

In [ ]:
df.dropna(how = 'any').shape

In [ ]:
df.duplicated().sum()

In [ ]:
df.drop_duplicates(subset = ['Product_ID', 'Gender', 'Age', 'Occupation', 'City_Category', 'Stay_In_Current_City_Years', 
                             'Marital_Status', 'Product_Category_1', 'Product_Category_2', 'Product_Category_3', 'Purchase']).shape

In [ ]:
df.info()

In [ ]:
df.columns

In [ ]:
df['Product_Category_2'] = df['Product_Category_2'].fillna(df['Product_Category_2'].mode()[0])
df['Product_Category_3'] = df['Product_Category_3'].fillna(df['Product_Category_3'].mode()[0])

In [ ]:
df.isnull().sum()

In [ ]:
df.nunique()

In [ ]:
df.apply(lambda x: x.dtype)

In [ ]:
df.columns.to_series().groupby(df.dtypes).groups

In [ ]:
round((df.apply(lambda x:x.isnull().sum())/len(df))*100,2)

In [ ]:
total_miss = df.isnull().sum()
perc_miss = total_miss/df.isnull().count()*100

missing_data = pd.DataFrame({'Total missing':total_miss,'% missing':perc_miss})

missing_data.sort_values(by='Total missing',ascending=False).head(3)

In [ ]:
(df.isnull().sum()/len(df))*100

In [ ]:
# find the unique values from categorical features
for col in df.select_dtypes(include='object').columns:
    print(col)
    print(df[col].unique())

In [ ]:
for column in df.columns:
    print(column,df[column].nunique())

In [ ]:
categorical_features = [feature for feature in df.columns if ((df[feature].dtypes=='O') & (feature not in ['deposit']))]
categorical_features

In [ ]:
for feature in categorical_features:
    print('The feature is {} and number of categories are {}'.format(feature,len(df[feature].unique())))

In [ ]:
numerical_data = df.select_dtypes(include=np.number) # select_dtypes selects data with numeric features
numerical_col = numerical_data.columns 

print("Numeric Features:")
print(numerical_data.head())
print("===="*20)

In [ ]:
categorical_data = df.select_dtypes(exclude=np.number) # we will exclude data with numeric features
categorical_col = categorical_data.columns                          # we will store the categorical features in a variable

print("Categorical Features:")
print(categorical_data.head())
print("===="*20)

In [ ]:
### numerical 
numerical_cols = list(df.select_dtypes(exclude=['object']))
numerical_cols

In [ ]:
### categorical
categorical_cols = list(df.select_dtypes(include=['object']))
categorical_cols

In [ ]:
#Check target label split over categorical features and find the count
for categorical_feature in categorical_features:
    print(df.groupby(['Purchase',categorical_feature]).size())

In [ ]:
# list of numerical variables
numerical_features = [feature for feature in df.columns if ((df[feature].dtypes != 'O') & (feature not in ['y']))]
print('Number of numerical variables: ', len(numerical_features))

# visualise the numerical variables
df[numerical_features].head()

In [ ]:
#Discrete Numerical Features
discrete_feature=[feature for feature in numerical_features if len(df[feature].unique())<25]
print("Discrete Variables Count: {}".format(len(discrete_feature)))

In [ ]:
#Continuous Numerical Features
continuous_features=[feature for feature in numerical_features if feature not in discrete_feature+['deposit']]
print("Continuous feature Count: {}".format(len(continuous_features)))

In [ ]:
cols_with_missing = [col for col in df.columns 
                                 if df[col].isnull().any()]
cols_with_missing

In [ ]:
df.describe()

In [ ]:
df.corr()

In [ ]:
plt.figure(figsize=(15,10))
sns.heatmap(df.corr(),annot=True)

In [ ]:
sns.pairplot(df)

In [ ]:
df.hist(figsize=(15,15))
plt.show()

In [ ]:
def bar_plot(variable):
    var = df[variable]
    varValue = var.value_counts()
    plt.figure(figsize=(15,3))
    plt.bar(varValue.index, varValue,color=['#00008b','#00e5ee','#cd1076', '#008080','#cd5555','red','blue',])
    plt.xticks(varValue.index, varValue.index.values)
    plt.ylabel("Frequency")
    plt.title(variable)
    
    plt.show()
    print("{}: \n {}".format(variable,varValue))

In [ ]:
categorical_cols = ['Gender', 'Age', 'City_Category', 'Stay_In_Current_City_Years']
for c in categorical_cols:
    bar_plot(c)

In [ ]:
categorcial_variables = ['Gender', 'Age', 'City_Category', 'Stay_In_Current_City_Years']
for col in categorcial_variables:
    plt.figure(figsize=(10,4))
    sns.barplot(df[col].value_counts().values, df[col].value_counts().index)
    plt.title(col)
    plt.tight_layout()

In [ ]:
columns = ['Gender','Age','City_Category','Stay_In_Current_City_Years','Marital_Status']
fig = plt.figure(figsize = (30, 20))
fig.subplots_adjust(hspace = 0.4, wspace = 0.4)
for i in range(1, (len(columns)**2)+1):
    ax = fig.add_subplot(len(columns), len(columns), i)
    ax = sns.countplot(df[columns[int((i-1)/len(columns))]],hue = columns[((i-1)%len(columns))],data = df)
    ax = plt.legend(loc = 'best')

plt.show()

In [ ]:
columns = ['Age','Gender','Occupation','City_Category','Stay_In_Current_City_Years','Marital_Status','Product_Category_1','Product_Category_2','Product_Category_3']
fig = plt.figure(figsize = (30, 20))
fig.subplots_adjust(hspace = 0.4, wspace = 0.4)
for i in range(1, (len(columns))+1):
    temp = df[['User_ID',columns[i-1],'Purchase']].groupby(['User_ID',columns[i-1]]).agg('sum').reset_index().sort_values('Purchase',ascending=False)
    temp = temp.sort_values(columns[i-1]) 
    ax = fig.add_subplot(3, 3, i)
    ax = sns.boxplot(x = columns[i-1],y = np.log(temp.Purchase),data = temp)

In [ ]:
columns = ['Age','Gender','Occupation','City_Category','Stay_In_Current_City_Years','Marital_Status']
fig = plt.figure(figsize=(20, 10))
fig.subplots_adjust(hspace = 0.4, wspace = 0.4)
for i in range(1, (len(columns))+1):
    label = []
    temp = df[['User_ID',columns[i-1],'Purchase']].groupby(['User_ID',columns[i-1]]).agg('sum').reset_index().sort_values('Purchase',ascending = False)
    ax = fig.add_subplot(2, 3, i)
    for j in list(df[columns[i-1]].unique()):
        hist_data = temp[temp[columns[i-1]]==j]
        ax = sns.distplot(np.log(hist_data.Purchase),hist=False,label = j)
        label.append(j)

In [ ]:
def Count_categorcial_variables(df):
    categorcial_variables = df.select_dtypes(include=['object']).columns.tolist()
    #fig = plt.figure(figsize=(14, 18))

    for index, col in enumerate(categorcial_variables):
        print("------------",col," value counts---------------------")
        print(df[col].value_counts())
        
        #fig.add_subplot(3, 2, index+1)
        #dataframe[col].value_counts()[:20].plot(kind='bar', title=col, color = "royalblue")
        #plt.tight_layout()
        
    print("\n\n------------Number of categories in each columns---------------------")
    for i in categorcial_variables:
        a = df[i].unique()
        print("There are {} categories in {}".format(len(a),i))
Count_categorcial_variables(df)

In [ ]:
# Info about products
print('Number of products:',df['Product_ID'].nunique())
print('Number of categories:',df['Product_Category_1'].unique().max())
print('Highest and lowest purchase:',
      df['Purchase'].max(),',',df['Purchase'].min())

In [ ]:
# Info about shoppers
print('Number of shoppers:',df['User_ID'].nunique())
print('Years in city:',df['Stay_In_Current_City_Years'].unique())
print('Age Groups:',df['Age'].unique())

In [ ]:
count_m = df[df['Gender']=='M'].count()[0]
count_f = df[df['Gender']=='F'].count()[0]

In [ ]:
print('Number of Male Clients:',count_m)
print('Number of Female Clients:',count_f)

In [ ]:
print('Female Purchases:',round(df[df['Gender']=='F']['Purchase'].sum()/count_f,3))
print('Male Purchases:',round(df[df['Gender']=='M']['Purchase'].sum()/count_m,3))

# Univariate Analaysis

In [ ]:
df['Gender'].value_counts()

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(y = df['Gender'])

In [ ]:
plt.figure(figsize=(10,10))
df['Gender'].value_counts().plot.pie(autopct="%0.2f%%")

In [ ]:
df['Age'].value_counts()

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(y = df['Age'])

In [ ]:
plt.figure(figsize=(10,10))
df['Age'].value_counts().plot.pie(autopct="%0.2f%%")

In [ ]:
df['Occupation'].value_counts()

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(y = df['Occupation'])

In [ ]:
plt.figure(figsize=(15,10))
df['Occupation'].value_counts().plot.pie(autopct="%0.2f%%")

In [ ]:
df['City_Category'].value_counts()

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(y = df['City_Category'])

In [ ]:
plt.figure(figsize=(15,10))
df['City_Category'].value_counts().plot.pie(autopct="%0.2f%%")

In [ ]:
df['Stay_In_Current_City_Years'].value_counts()

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(y = df['Stay_In_Current_City_Years'])

In [ ]:
plt.figure(figsize=(15,10))
df['Stay_In_Current_City_Years'].value_counts().plot.pie(autopct="%0.2f%%")

In [ ]:
df['Marital_Status'].value_counts()

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(y = df['Marital_Status'])

In [ ]:
plt.figure(figsize=(15,10))
df['Marital_Status'].value_counts().plot.pie(autopct="%0.2f%%")

In [ ]:
df['Product_Category_1'].value_counts()

In [ ]:
plt.figure(figsize=(10,10))
sns.countplot(y = df['Product_Category_1'])

In [ ]:
plt.figure(figsize=(20,15))
df['Product_Category_1'].value_counts().plot.pie(autopct="%0.2f%%")

In [ ]:
df['Product_Category_2'].value_counts()

In [ ]:
plt.figure(figsize=(10,10))
sns.countplot(y = df['Product_Category_2'])

In [ ]:
plt.figure(figsize=(20,15))
df['Product_Category_2'].value_counts().plot.pie(autopct="%0.2f%%")

In [ ]:
df['Product_Category_3'].value_counts()

In [ ]:
plt.figure(figsize=(10,10))
sns.countplot(y = df['Product_Category_3'])

In [ ]:
plt.figure(figsize=(20,15))
df['Product_Category_3'].value_counts().plot.pie(autopct="%0.2f%%")

In [ ]:
df['Purchase'].value_counts()

In [ ]:
df["Gender"].replace("F",0,inplace=True)
df["Gender"].replace('M',1,inplace=True)
df.head()

In [ ]:
df["Age"].replace('0-17','17',inplace=True)
df["Age"].replace('55+','56',inplace=True)
df["Age"].replace('26-35','30',inplace=True)
df["Age"].replace('46-50','48',inplace=True)
df["Age"].replace('51-55','53',inplace=True)
df["Age"].replace('36-45','40',inplace=True)
df["Age"].replace('18-25','21',inplace=True)
df.head()

In [ ]:
df["City_Category"].replace('A',3,inplace=True)
df["City_Category"].replace('B',2,inplace=True)
df['City_Category'].replace('C',1,inplace=True)
df.head()

In [ ]:
df["Stay_In_Current_City_Years"].replace('0',4,inplace=True)
df["Stay_In_Current_City_Years"].replace('1',3,inplace=True)
df['Stay_In_Current_City_Years'].replace('2',2,inplace=True)
df["Stay_In_Current_City_Years"].replace('3',1,inplace=True)
df['Stay_In_Current_City_Years'].replace('4+',0,inplace=True)
df.head()

# Dist Plot

In [ ]:
plt.figure(figsize = (10, 10))
sns.distplot(x = df['Occupation'])

In [ ]:
plt.figure(figsize = (10, 10))
sns.distplot(x = df['Product_Category_1'])

In [ ]:
plt.figure(figsize = (10, 10))
sns.distplot(x = df['Product_Category_2'])

In [ ]:
plt.figure(figsize = (10, 10))
sns.distplot(x = df['Product_Category_3'])

In [ ]:
plt.figure(figsize = (10, 10))
sns.distplot(x = df['Purchase'])

# Bivariate Analysis

# Joint Plot

In [ ]:
sns.jointplot(x = "Occupation", y = "Purchase", data = df)

In [ ]:
sns.jointplot(x = "Gender", y = "Purchase", data = df)

In [ ]:
sns.jointplot(x = "Age", y = "Purchase", data = df)

In [ ]:
sns.jointplot(x = "City_Category", y = "Purchase", data = df)

In [ ]:
sns.jointplot(x = "Stay_In_Current_City_Years", y = "Purchase", data = df)

# Bar Plot

In [ ]:
sns.barplot('Gender', 'Purchase', data = df)

In [ ]:
plt.figure(figsize = (10,10))
sns.barplot('Age', 'Purchase', data = df)

In [ ]:
plt.figure(figsize = (10,10))
sns.barplot('City_Category', 'Purchase', data = df)

In [ ]:
plt.figure(figsize = (10,10))
sns.barplot('Stay_In_Current_City_Years', 'Purchase', data = df)

In [ ]:
plt.figure(figsize = (10,10))
sns.barplot('Marital_Status', 'Purchase', data = df)

In [ ]:
plt.figure(figsize = (10,10))
sns.barplot('Product_Category_1', 'Purchase', data = df)

In [ ]:
plt.figure(figsize = (10,10))
sns.barplot('Product_Category_2', 'Purchase', data = df)

In [ ]:
plt.figure(figsize = (10,10))
sns.barplot('Product_Category_3', 'Purchase', data = df)

In [ ]:
plt.figure(figsize = (10,10))
sns.barplot('Product_Category_1', 'Occupation', data = df)

In [ ]:
plt.figure(figsize = (10,10))
sns.barplot('Product_Category_2', 'Occupation', data = df)

In [ ]:
plt.figure(figsize = (10,10))
sns.barplot('Product_Category_3', 'Occupation', data = df)

# Box Plot

In [ ]:
plt.figure(figsize=(15,8))
sns.boxplot(df['Product_Category_1'],df['Purchase'])

In [ ]:
plt.figure(figsize=(15,8))
sns.boxplot(df['Product_Category_2'],df['Purchase'])

In [ ]:
plt.figure(figsize=(15,8))
sns.boxplot(df['Product_Category_3'],df['Purchase'])

In [ ]:
plt.figure(figsize=(15,8))
sns.boxplot(df['Age'],df['Purchase'])

In [ ]:
plt.figure(figsize=(15,8))
sns.boxplot(df['City_Category'],df['Purchase'])

In [ ]:
plt.figure(figsize=(15,8))
sns.boxplot(df['Stay_In_Current_City_Years'],df['Purchase'])

In [ ]:
plt.figure(figsize = (15,8))
sns.boxplot(x = 'Age',y = 'Purchase',data = df,hue = 'Gender')

In [ ]:
plt.figure(figsize = (15,8))
sns.boxplot(x = 'Occupation',y = 'Purchase',data = df,hue = 'Gender')

In [ ]:
plt.figure(figsize = (15,8))
sns.boxplot(x = 'City_Category',y = 'Purchase',data = df,hue = 'Gender')

In [ ]:
plt.figure(figsize = (15,8))
sns.boxplot(x = 'Stay_In_Current_City_Years',y = 'Purchase',data = df,hue = 'Gender')

In [ ]:
plt.figure(figsize = (15,8))
sns.boxplot(x = 'Marital_Status',y = 'Purchase',data = df,hue = 'Gender')

In [ ]:
plt.figure(figsize = (15,8))
sns.boxplot(x = 'Product_Category_1',y = 'Purchase',data = df,hue = 'Gender')

In [ ]:
plt.figure(figsize = (15,8))
sns.boxplot(x = 'Product_Category_2',y = 'Purchase',data = df,hue = 'Gender')

In [ ]:
plt.figure(figsize = (15,8))
sns.boxplot(x = 'Product_Category_3',y = 'Purchase',data = df,hue = 'Gender')

# Count Plot

In [ ]:
plt.figure(figsize = (10,10))
ax = sns.countplot('Age', hue = 'Gender',data = df, order = ['17', '21', '30', '40', '48', '53', '56']);

In [ ]:
sns.countplot('Gender', data = df, hue = 'Age')
plt.show()

In [ ]:
sns.countplot('Marital_Status', data = df, hue = 'Age')
plt.show()

In [ ]:
plt.figure(figsize = (15,8))
sns.countplot('Occupation', data = df, hue = 'Age')

In [ ]:
plt.figure(figsize = (15,8))
sns.countplot('Occupation', data = df, hue = 'Product_Category_1')

In [ ]:
plt.figure(figsize = (20, 7))
ax = sns.countplot('Gender', hue = 'Product_Category_1', data = df);

In [ ]:
ax = sns.countplot('Gender', hue = 'Marital_Status', data = df);

In [ ]:
sns.countplot('Gender', data = df);

In [ ]:
sns.countplot('Age', data = df)

In [ ]:
sns.countplot('Occupation', data = df)

In [ ]:
sns.countplot('City_Category', data = df)

In [ ]:
sns.countplot('Stay_In_Current_City_Years', data = df)

In [ ]:
sns.countplot('Marital_Status', data = df)

In [ ]:
plt.figure(figsize = (10,10))
sns.countplot('Product_Category_1', data = df)

In [ ]:
plt.figure(figsize = (10,10))
sns.countplot('Product_Category_2', data = df)

In [ ]:
plt.figure(figsize = (10,10))
sns.countplot('Product_Category_3', data = df)

In [ ]:
Age = df['Age'].value_counts()

plt.style.use('default')
plt.figure(figsize = (7, 5))
squarify.plot(sizes = Age.values, label = Age.index, value = Age.values)
plt.title('Age Distribution', fontdict = {'fontname' : 'Monospace', 'fontsize' : 20, 'fontweight' : 'bold'})
plt.show()

In [ ]:
Gender = df['Gender'].value_counts()

plt.style.use('default')
plt.figure(figsize = (5, 5))
squarify.plot(sizes = Gender.values, label = Gender.index, value = Gender.values)
plt.title('Gender Distribution', fontdict = {'fontname' : 'Monospace', 'fontsize' : 20, 'fontweight' : 'bold'})
plt.show()

In [ ]:
Occupation = df['Occupation'].value_counts()

plt.style.use('default')
plt.figure(figsize = (10, 7))
squarify.plot(sizes = Occupation.values, label = Occupation.index, value = Occupation.values)
plt.title('Occupation Distribution', fontdict = {'fontname' : 'Monospace', 'fontsize' : 20, 'fontweight' : 'bold'})
plt.show()

In [ ]:
City_Category = df['City_Category'].value_counts()

plt.style.use('default')
plt.figure(figsize = (7, 5))
squarify.plot(sizes = City_Category.values, label = City_Category.index, value = City_Category.values)
plt.title('City Category Distribution', fontdict = {'fontname' : 'Monospace', 'fontsize' : 20, 'fontweight' : 'bold'})
plt.show()

In [ ]:
stay = df['Stay_In_Current_City_Years'].value_counts()

plt.style.use('default')
plt.figure(figsize = (7, 5))
squarify.plot(sizes = stay.values, label = stay.index, value = stay.values)
plt.title('Stay in current city distribution', fontdict = {'fontname' : 'Monospace', 'fontsize' : 20, 'fontweight' : 'bold'})
plt.show()

In [ ]:
Marital_Status = df['Marital_Status'].value_counts()

plt.style.use('default')
plt.figure(figsize = (7, 5))
squarify.plot(sizes = Marital_Status.values, label = Marital_Status.index, value = Marital_Status.values)
plt.title('Marital Status Distribution', fontdict = {'fontname' : 'Monospace', 'fontsize' : 20, 'fontweight' : 'bold'})
plt.show()

In [ ]:
Product_Category_1 = df['Product_Category_1'].value_counts()

plt.style.use('default')
plt.figure(figsize = (15, 15))
squarify.plot(sizes = Product_Category_1.values, label = Product_Category_1.index, value = Product_Category_1.values)
plt.title('Product Category 1 Distribution', fontdict = {'fontname' : 'Monospace', 'fontsize' : 20, 'fontweight' : 'bold'})
plt.show()

In [ ]:
Product_Category_2 = df['Product_Category_2'].value_counts()

plt.style.use('default')
plt.figure(figsize = (15, 15))
squarify.plot(sizes = Product_Category_2.values, label = Product_Category_2.index, value = Product_Category_2.values)
plt.title('Product Category  Distribution', fontdict = {'fontname' : 'Monospace', 'fontsize' : 20, 'fontweight' : 'bold'})
plt.show()

In [ ]:
Product_Category_3 = df['Product_Category_3'].value_counts()

plt.style.use('default')
plt.figure(figsize = (15, 15))
squarify.plot(sizes = Product_Category_3.values, label = Product_Category_3.index, value = Product_Category_3.values)
plt.title('Product Category 3 Distribution', fontdict = {'fontname' : 'Monospace', 'fontsize' : 20, 'fontweight' : 'bold'})
plt.show()

# Violin Plot

In [ ]:
plt.figure(figsize = (10, 7))
ax = sns.violinplot(x = "Gender", y = "Purchase", data = df)

In [ ]:
plt.figure(figsize = (10, 7))
ax = sns.violinplot(x = "Age", y = "Purchase", data = df)

In [ ]:
plt.figure(figsize = (10, 7))
ax = sns.violinplot(x = 'Occupation', y = "Purchase", data = df)

In [ ]:
plt.figure(figsize = (10, 7))
ax = sns.violinplot(x = "City_Category", y = "Purchase", data = df)

In [ ]:
plt.figure(figsize = (10, 7))
ax = sns.violinplot(x = "Stay_In_Current_City_Years", y = "Purchase", data = df)

In [ ]:
plt.figure(figsize = (10, 7))
ax = sns.violinplot(x = "Marital_Status", y = "Purchase", data = df)

In [ ]:
plt.figure(figsize = (10, 7))
ax = sns.violinplot(x = "Product_Category_2", y = "Purchase", data = df)

In [ ]:
plt.figure(figsize = (10, 7))
ax = sns.violinplot(x = "Product_Category_3", y = "Purchase", data = df)

In [ ]:
plt.figure(figsize = (10, 7))
ax = sns.violinplot(x = "Product_Category_3", y = "City_Category", data = df)

In [ ]:
plt.figure(figsize = (10, 7))
ax = sns.violinplot(x = "Product_Category_2", y = "City_Category", data = df)

In [ ]:
plt.figure(figsize = (10, 7))
ax = sns.violinplot(x = "Product_Category_1", y = "City_Category", data = df)

In [ ]:
plt.figure(figsize = (10, 7))
ax = sns.violinplot(x = "City_Category", y = "Purchase", hue = 'Gender',data = df)

In [ ]:
plt.figure(figsize = (10, 7))
ax = sns.violinplot(x = "City_Category", y = "Purchase", hue = 'Age',data = df)

In [ ]:
plt.figure(figsize = (10, 7))
ax = sns.violinplot(x = "Marital_Status", y = "Purchase", hue = 'Age',data = df)

In [ ]:
plt.figure(figsize = (10, 7))
ax = sns.violinplot(x = "Marital_Status", y = "Purchase", hue = 'Gender',data = df)

In [ ]:
plt.figure(figsize = (10, 7))
ax = sns.violinplot(x = "City_Category", y = "Purchase", hue = 'Marital_Status',data = df)

In [ ]:
plt.figure(figsize = (10, 7))
ax = sns.violinplot(x = "Marital_Status", y = "Purchase", hue = 'Stay_In_Current_City_Years',data = df)

# Point Plot

In [ ]:
figures, axes = plt.subplots(2, 2, figsize=(16, 14))

ax = sns.pointplot(y = "Product_Category_1", x = "City_Category", hue = 'Age',data = df, ax = axes[0,0])
ax = sns.pointplot(y = "Product_Category_2", x = "City_Category", hue = 'Age',data = df, ax = axes[0,1])
ax = sns.pointplot(y = "Product_Category_3", x = "City_Category", hue = 'Age',data = df , ax = axes[1,0])
ax = sns.pointplot(y = "Purchase", x = "City_Category", hue = 'Age',data = df, ax = axes[1,1])

plt.show()

# Pivot Table

In [ ]:
table = pd.pivot_table(data = df,index = ['Gender'])
table

In [ ]:
table = pd.pivot_table(data = df,index = ['Age'])
table

In [ ]:
table = pd.pivot_table(data = df,index = ['Occupation'])
table

In [ ]:
table = pd.pivot_table(data = df,index = ['City_Category'])
table

In [ ]:
table = pd.pivot_table(data = df,index = ['Stay_In_Current_City_Years'])
table

In [ ]:
table = pd.pivot_table(data = df,index = ['Marital_Status'])
table

In [ ]:
table = pd.pivot_table(data = df,index = ['Product_Category_1'])
table

In [ ]:
table = pd.pivot_table(data = df,index = ['Product_Category_2'])
table

In [ ]:
table = pd.pivot_table(data = df,index = ['Product_Category_3'])
table

In [ ]:
table = pd.pivot_table(data = df,index = ['Gender','Age'])
table

In [ ]:
table = pd.pivot_table(data = df,index = ['Gender','Occupation'])
table

In [ ]:
table = pd.pivot_table(data = df,index = ['Gender','City_Category'])
table

In [ ]:
table = pd.pivot_table(data = df,index = ['Gender','Stay_In_Current_City_Years'])
table

In [ ]:
table = pd.pivot_table(data = df,index = ['Gender','Marital_Status'])
table

In [ ]:
table = pd.pivot_table(data = df,index = ['Gender','Product_Category_1'])
table

In [ ]:
table = pd.pivot_table(data = df,index = ['Gender','Product_Category_2'])
table

In [ ]:
table = pd.pivot_table(data = df,index = ['Gender','Product_Category_3'])
table

# Cross Table

In [ ]:
pd.crosstab(df['Gender'],df['Age']).style.background_gradient(cmap = 'winter')

In [ ]:
pd.crosstab(df['City_Category'],df['Gender']).style.background_gradient(cmap = 'spring')

In [ ]:
pd.crosstab(df['Stay_In_Current_City_Years'],df['Gender']).style.background_gradient(cmap = 'autumn')

In [ ]:
pd.crosstab(df['Marital_Status'],df['Gender']).style.background_gradient(cmap = 'cool')

In [ ]:
pd.crosstab(df['Product_Category_1'],df['Gender']).style.background_gradient(cmap = 'bwr')

In [ ]:
pd.crosstab(df['City_Category'],df['Age']).style.background_gradient(cmap = 'spring')

In [ ]:
pd.crosstab(df['City_Category'],df['Occupation']).style.background_gradient(cmap = 'cool')

In [ ]:
pd.crosstab(df['Stay_In_Current_City_Years'],df['Occupation']).style.background_gradient(cmap = 'cool')

# Line Plot

In [ ]:
plt.figure(figsize = (20,7))
sns.lineplot(x = df['City_Category'],y = df['Marital_Status'],hue = df['Gender'])

In [ ]:
plt.figure(figsize = (20,7))
sns.lineplot(x = df['City_Category'],y = df['Purchase'],hue = df['Gender'])

In [ ]:
plt.figure(figsize = (20,7))
sns.lineplot(x = df['City_Category'],y = df['Age'],hue = df['Gender'])

In [ ]:
plt.figure(figsize = (20,7))
sns.lineplot(x = df['Occupation'],y = df['City_Category'],hue = df['Age'])

In [ ]:
plt.figure(figsize = (20,7))
sns.lineplot(x = df['Occupation'],y = df['City_Category'],hue = df['Stay_In_Current_City_Years'])

In [ ]:
plt.figure(figsize = (20,7))
sns.lineplot(x = df['Occupation'],y = df['City_Category'],hue = df['Marital_Status'])

In [ ]:
plt.figure(figsize = (20,7))
sns.lineplot(x = df['Product_Category_1'],y = df['City_Category'],hue = df['Age'])

# Boxen Plot

In [ ]:
plt.figure(figsize = (20,7))
sns.boxenplot(x = df['City_Category'],y = df['Purchase'], hue = df['Gender'])

In [ ]:
plt.figure(figsize = (20,7))
sns.boxenplot(x = df['City_Category'],y = df['Purchase'], hue = df['Age'])

In [ ]:
plt.figure(figsize = (20,7))
sns.boxenplot(x = df['City_Category'],y = df['Purchase'], hue = df['Marital_Status'])

In [ ]:
plt.figure(figsize = (20,7))
sns.boxenplot(x = df['City_Category'],y = df['Purchase'], hue = df['Stay_In_Current_City_Years'])

In [ ]:
plt.figure(figsize = (20,7))
sns.boxenplot(x = df['Age'],y = df['Purchase'], hue = df['Stay_In_Current_City_Years'])

In [ ]:
plt.figure(figsize = (20,7))
sns.boxenplot(x = df['Gender'],y = df['Purchase'], hue = df['Stay_In_Current_City_Years'])

In [ ]:
plt.figure(figsize = (20,7))
sns.boxenplot(x = df['Occupation'],y = df['Purchase'], hue = df['Stay_In_Current_City_Years'])

In [ ]:
plt.figure(figsize = (20,7))
sns.boxenplot(x = df['Marital_Status'],y = df['Purchase'], hue = df['Stay_In_Current_City_Years'])

# GroupBy Plot

In [ ]:
print(df.groupby(['City_Category','Age'])['Gender'].mean())

In [ ]:
print(df.groupby(['City_Category','Age'])['Marital_Status'].mean())

In [ ]:
print(df.groupby(['City_Category','Gender'])['Marital_Status'].mean())

In [ ]:
df = df.dropna()
#print(df)
df.head()

In [ ]:
df.dropna(inplace = True)
df.isnull().sum()

# Training And Testing Data

In [ ]:
#Classifiers
X = df.loc[:, ['User_ID', 'Gender', 'Age', 'Occupation', 'City_Category', 'Stay_In_Current_City_Years', 'Marital_Status', 
               'Product_Category_1', 'Product_Category_2', 'Product_Category_3']]
X.head()

In [ ]:
Y = df.loc[:, ['Purchase']]
Y.head()

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.25, random_state = 42, shuffle = True)

# Linear Regression

In [ ]:
regressor = LinearRegression()  
regressor.fit(X_train, Y_train) #training the algorithm
#To retrieve the intercept:
print(regressor.intercept_)

#For retrieving the slope:
print(regressor.coef_)

In [ ]:
Y_pred = regressor.predict(X_test)
print('Mean Absolute Error:', metrics.mean_absolute_error(Y_test, Y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(Y_test, Y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(Y_test, Y_pred)))

In [ ]:
# Model initialization
regression_model = LinearRegression()
# Fit the data(train the model)
regression_model.fit(X, Y)
# Predict
Y_pred = regression_model.predict(X)

# model evaluation
rmse = mean_squared_error(Y, Y_pred)
r2 = r2_score(Y, Y_pred)

# printing values
print('Slope:' ,regression_model.coef_)
print('Intercept:', regression_model.intercept_)
print('Root mean squared error: ', rmse)
print('R2 score: ', r2)

In [ ]:
import statsmodels.api as sm

X = np.random.rand(100)
Y = X + np.random.rand(100)*0.1

results = sm.OLS(Y,sm.add_constant(X)).fit()

print(results.summary())

plt.scatter(X,Y)

X_plot = np.linspace(0,1,100)
plt.plot(X_plot, X_plot*results.params[0] + results.params[1])

plt.show()